In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"
import openai
openai.api_key = "sk-"

In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [4]:
from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader('assets/AndrewHuberman/sleep', filename_as_id=True).load_data()

In [5]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

/Users/jana/work/LlamaIndexLangChain/llamaindexscripts/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu
Use pytorch device: cpu


In [8]:
from llama_index.vector_stores import ChromaVectorStore
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.storage.storage_context import StorageContext

import chromadb

db = chromadb.PersistentClient(path="./storage/chroma")
chroma_collection = db.get_or_create_collection("andrew_sleeping_database")

service_context = ServiceContext.from_defaults(embed_model=embed_model)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, service_context=service_context)

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.72it/s]


In [9]:
query_engine = index.as_query_engine()
response = query_engine.query("How does sun light affect sleep?")
print(response)

Batches: 100%|██████████| 1/1 [00:00<00:00, 29.68it/s]


Sunlight affects sleep by influencing the circadian rhythm, which regulates the sleep-wake cycle. Exposure to sunlight in the morning helps to synchronize the internal body clock and promotes alertness and wakefulness during the day. Sunlight also helps to suppress the production of melatonin, a hormone that promotes sleep, making it easier to stay awake and alert. On the other hand, exposure to bright light in the evening or at night can disrupt the circadian rhythm and make it more difficult to fall asleep. It is important to maintain a regular sleep schedule and limit exposure to bright light, especially in the evening, to promote healthy sleep patterns.
